<a href="https://colab.research.google.com/github/shirinpirouzkhaah/Impacts-of-code-review-smells/blob/notebook-conversion/Impact_of_Code_Review_Smells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing the Impact of Code Review Smells

Welcome to the analysis notebook for this paper.

Let's get you up and running!

First things first: There cannot be any analysis until the data files are there. Let's fetch the data files.

This can be accomplished by **running the code in the next cell!**

In [18]:
# this cell for setting some initial configuration options
WORKING_DIRECTORY = "/content/drive/MyDrive/oss project data/code_review_data"
DATA_FILES_LIST = [
                   "libreoffice.zip",
                   "eclipse.zip",
                   "qt.zip",
                   "wireshark.zip"
]

In [20]:
# this cell helps you extract the code review data in order to be able to use them
import zipfile
for data_file in DATA_FILES_LIST:
  with zipfile.ZipFile(WORKING_DIRECTORY + '/' + data_file, "r") as zip_ref:
    zip_ref.extractall('./')



In [1]:
# this cell will help you import the file you want to analyze the data for
from datetime import datetime
import pandas as pd
from scipy.stats import norm 
import matplotlib.pyplot as plt

# constants: general
NUM_OF_SECONDS_IN_ONE_DAY = 86400
LOG_MESSAGE_TRIGGER_INDEX = 1000

# read json project code review data
cr_data_raw_json = pd.read_json (r'/content/libreoffice_cr_data.json', lines=True) 



In [2]:
# this cell normalizes the top level data imported from the json file
cr_data_normalized_json = pd.json_normalize(cr_data_raw_json['data'])
cr_data_normalized_json = cr_data_normalized_json[cr_data_normalized_json.status != 'NEW']
cr_data_normalized_json = cr_data_normalized_json.reset_index()

In [3]:
# this cell normalizes the patchset and comments data that will be used in further analysis

# comments
cr_data_comments = []
for index,row in cr_data_normalized_json.iterrows():
    cr_data_comments.append(pd.json_normalize(row['comments']))
    if index % LOG_MESSAGE_TRIGGER_INDEX == 0:
        print(f'{index} comments objects normalized!')


# patchsets
cr_data_patchsets = []
for index,row in cr_data_normalized_json.iterrows():
    cr_data_patchsets.append(pd.json_normalize(row['patchSets']))
    if index % LOG_MESSAGE_TRIGGER_INDEX == 0:
        print(f'{index} patchsets objects normalized!')



0 comments objects normalized!
1000 comments objects normalized!
2000 comments objects normalized!
3000 comments objects normalized!
4000 comments objects normalized!
5000 comments objects normalized!
6000 comments objects normalized!
7000 comments objects normalized!
8000 comments objects normalized!
9000 comments objects normalized!
10000 comments objects normalized!
11000 comments objects normalized!
12000 comments objects normalized!
13000 comments objects normalized!
14000 comments objects normalized!
15000 comments objects normalized!
16000 comments objects normalized!
17000 comments objects normalized!
18000 comments objects normalized!
19000 comments objects normalized!
20000 comments objects normalized!
21000 comments objects normalized!
22000 comments objects normalized!
23000 comments objects normalized!
24000 comments objects normalized!
25000 comments objects normalized!
26000 comments objects normalized!
27000 comments objects normalized!
28000 comments objects normalized

In [ ]:
# this cell is for preprocessing some of the data so we can analyze it later
from sys import breakpoint

pr_completion_duration = []
pr_num_of_iterations = []

# for each merged pull request
for index in range (len(cr_data_comments)): 
    num_of_iterations = 0
    current_pr_comments = cr_data_comments[index]
    last_comment = current_pr_comments.iloc[-1] # last operation of reviewing a PR
    num_of_comments = len(current_pr_comments.index)-1 # number of operations in reviewing a PR
    
    # NaN check
    if (last_comment["reviewer.name"] == last_comment["reviewer.name"]  ):
        # Bot check and drop
        if (last_comment["reviewer.name"].find("Bot") != -1): 
            current_pr_comments.drop(current_pr_comments.tail(1).index,inplace =True) 
    
    for indx in range (len(current_pr_comments)):
        # occurrence of 'Uploaded' indicates the start of another iteration
        if (current_pr_comments.iloc[indx]["message"].find('Uploaded') != -1):
            num_of_iterations = num_of_iterations + 1

    # substract time stamp of last review operation from first review operation         
    current_pr_time_difference = current_pr_comments.tail(1)["timestamp"].iloc[0] - current_pr_comments.head(1)["timestamp"].iloc[0]
    breakpoint()
    current_pr_time_difference = current_pr_time_difference / NUM_OF_SECONDS_IN_ONE_DAY# convert seconds to days 
    pr_completion_duration.append(current_pr_time_difference) # dataframe of time of completion of PR reviews 
    pr_num_of_iterations.append(num_of_iterations) # dataframe of iterations in each PR 

print(pr_num_of_iterations[1:5])
print(pr_completion_duration[1:5])

